In [1]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore
import requests
from bs4 import BeautifulSoup
import json
import openai
def get_model_by_dataset_name(dataset_name):
  return [e for e in openai.FineTune.list()["data"] if e["training_files"][0]["filename"] == dataset_name][0]
# If fine_tuned_model is null, it means you have to wait OpenAI process the fine-tuning

In [2]:
langame_client = LangameClient()

In [29]:
# TODO: use protobuf
p = {}
p["type"] = "TopicGeneralist"
p["template"] = """This is a conversation starter that leads to extremely profound conversations between people about "[TOPIC]".
Starter: What do you think of the theory that animals have emotions?
Starter: What is the role of philosophy in human society?
Starter: How does one recognize truth and knowledge? What are the ways to acquire it?
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 16, 15, 49, 26, 906587, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f4c13177610>)


In [22]:
p = {}
p["type"] = "TopicGeneralistFineTuned"
p["template"] = "[TOPIC]. ->"
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": get_model_by_dataset_name("generate-generalist-0.0.1.jsonl")["fine_tuned_model"],
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 30, 14, 29, 29, 759625, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f7db4610b80>)


In [14]:
for _ in range(10):
    langame_client.prompt_to_meme("knowledge", model="TopicGeneralist")

calling openai with {'id': '544wPT15TsrSuNtpAyUQ', 'prompt': 'This is a conversation starter that leads to extremely profound conversations between people about "knowledge".\nStarter: What do you think of the theory that animals have emotions?\nStarter: What is the role of philosophy in human society?\nStarter: How does one recognize truth and knowledge? What are the ways to acquire it?\nStarter:', 'parameters': {'topP': 1, 'temperature': 0.7, 'frequencyPenalty': 0.1, 'presencePenalty': 0.1, 'model': 'davinci', 'maxTokens': 150, 'stop': ['\\n', 'Starter:']}}
calling openai with {'id': '544wPT15TsrSuNtpAyUQ', 'prompt': 'This is a conversation starter that leads to extremely profound conversations between people about "knowledge".\nStarter: What do you think of the theory that animals have emotions?\nStarter: What is the role of philosophy in human society?\nStarter: How does one recognize truth and knowledge? What are the ways to acquire it?\nStarter:', 'parameters': {'temperature': 0.7

In [16]:
# TODO: use protobuf
p = {}
p["type"] = "IceBreakerGeneralist"
p["template"] = """This is conversation starters that leads to extremely personal discussions between people.
Starter: What is an interesting fact that few people know about you?
Starter: What is something that most people take for granted, but has a deeper meaning to you?
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 16, 15, 34, 18, 122873, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f4c131a0fd0>)


In [28]:
langame_client.prompt_to_meme_ice_breaker()

calling openai with {'id': 'oCccJ82pVND4znQx3zM5', 'prompt': 'This is conversation starters that leads to extremely personal discussions between people.\nStarter: What is an interesting fact that few people know about you?\nStarter: What is something that most people take for granted, but has a deeper meaning to you?\nStarter:', 'parameters': {'temperature': 0.7, 'maxTokens': 150, 'frequencyPenalty': 0.1, 'model': 'davinci', 'presencePenalty': 0.1, 'stop': ['\\n', 'Starter:'], 'topP': 1}}


(DatetimeWithNanoseconds(2021, 7, 16, 15, 35, 59, 312111, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f4c131a0dc0>)

In [3]:

completions = []
topics = ["philosophy", "physic", "mathematic", "biology", "transhumanism", "artificial intelligence", "book"]
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", topics)\
    .order_by(u"createdAt", direction=firestore.Query.DESCENDING)\
    .stream():
    completions.append({"topic": t.to_dict()["topic"]["content"], "meme": t.reference.parent.parent.get().to_dict()["content"]})
pprint(len(completions))


174


In [4]:
dataset = [{"prompt": f"{e['topic']} ->", "completion": f" {e['meme']}\n"} for e in completions]

In [6]:
import json
file_name = 'generate-generalist-0.0.1.jsonl'
with open(file_name, 'w') as outfile:
    for entry in dataset:
        json.dump(entry, outfile)
        outfile.write('\n')

In [7]:
!openai tools fine_tunes.prepare_data -f $file_name

Analyzing...

- Your file contains 174 prompt-completion pairs
- There are 2 duplicated prompt-completion pairs. These are rows: [12, 21]
- All prompts end with suffix ` ->`
- All completions end with suffix `\n`
` more than once. We suggest that you review your completions and add a unique ending

Based on the analysis we will perform the following actions:
- [Recommended] Remove 2 duplicate rows [Y/n]: ^C



In [9]:
import openai
f = openai.File.create(
  file=open(file_name),
  purpose="fine-tune"
)
f_t = openai.FineTune.create(
    training_file=f["id"],
    model="curie"
)


Upload progress:   0%|          | 0.00/35.8k [00:00<?, ?it/s]
Upload progress:  23%|██▎       | 8.19k/35.8k [00:00<00:00, 32.5kit/s]
Upload progress:  46%|████▌     | 16.4k/35.8k [00:00<00:00, 64.5kit/s]
Upload progress:  69%|██████▊   | 24.6k/35.8k [00:00<00:00, 96.2kit/s]
Upload progress: 100%|██████████| 35.8k/35.8k [00:00<00:00, 139kit/s]


<File file id=file-wgDEM5MCzKoqGfW8N5FCEL6U at 0x7f7db448c630> JSON: {
  "bytes": 35542,
  "created_at": 1627653912,
  "filename": "generate-generalist-0.0.1.jsonl",
  "id": "file-wgDEM5MCzKoqGfW8N5FCEL6U",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [15]:
model = get_model_by_dataset_name("generate-generalist-0.0.1.jsonl")
model

<FineTune fine-tune id=ft-5DIo5s9BIk7ZLwB8biEVnzHB at 0x7f7db4395e50> JSON: {
  "created_at": 1627653967,
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": 4,
    "learning_rate_multiplier": 0.1,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1,
    "use_packing": null
  },
  "id": "ft-5DIo5s9BIk7ZLwB8biEVnzHB",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-6iVquBDkjuvNsgZOGPJsaRAs",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 35542,
      "created_at": 1627653912,
      "filename": "generate-generalist-0.0.1.jsonl",
      "id": "file-wgDEM5MCzKoqGfW8N5FCEL6U",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1627653972,
  "user_id": "user-fs8ZlgenluyrbB3JQwgRc7O8",
  "validation_files": []
}

In [76]:
openai.Completion.create(
  model=model["fine_tuned_model"],
  prompt="Transhumanism. ->",
  max_tokens=100,
  stop=["\n"]
)

<OpenAIObject text_completion id=cmpl-3O9em9S5iLE0b0IXIJSxFb004GtuE at 0x7f560072c770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " Are there any changes in human evolution that we have not experienced yet in our lifetime, locations at which universal humanism is furthest from what we are used to, and perhaps will be even further away from some day in the future"
    }
  ],
  "created": 1626886004,
  "id": "cmpl-3O9em9S5iLE0b0IXIJSxFb004GtuE",
  "model": "curie:ft-beaumont-2021-07-21-16-19-54",
  "object": "text_completion"
}

In [ ]:
# Below = fine tunes

In [4]:
URL = "https://conversationstartersworld.com/philosophical-questions/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [12]:
philosophical_questions = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]

In [46]:
memes = []
for e in langame_client._firestore_client.collection("memes").stream():
    memes.append(e.get("content"))
memes = memes + [" ".join(e.split(" ")[1:]) for e in philosophical_questions[3:-5]]


In [38]:
URL = "https://conversationstartersworld.com/deep-conversation-topics/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
deep = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]

In [50]:
memes += [" ".join(e.split(" ")[1:]) for e in deep[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

In [51]:

URL = "https://conversationstartersworld.com/hypothetical-questions/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
hypothetical = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]

In [52]:
memes += [" ".join(e.split(" ")[1:]) for e in hypothetical[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

In [55]:
URL = "https://conversationstartersworld.com/open-ended-questions/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
open_ended = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]

In [58]:
memes += [" ".join(e.split(" ")[1:]) for e in open_ended[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

In [59]:

URL = "https://conversationstartersworld.com/closed-ended-questions-3/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
closed_ended = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]

In [60]:
memes += [" ".join(e.split(" ")[1:]) for e in closed_ended[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

In [66]:
URL = "https://conversationstartersworld.com/good-questions-to-ask/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
good_question_to_ask = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]

In [67]:
memes += [" ".join(e.split(" ")[1:]) for e in good_question_to_ask[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

In [68]:

URL = "https://conversationstartersworld.com/250-conversation-starters/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
ss = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]

In [69]:
memes += [" ".join(e.split(" ")[1:]) for e in ss[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

In [71]:
file_name = 'generate-generalist-0.0.2.jsonl'
with open(file_name, 'w') as outfile:
    for meme in memes:
        json.dump({
            "prompt": "",
            "completion": f" {meme}\n\n"
        }, outfile)
        outfile.write('\n')

In [78]:
import openai
f = openai.File.create(
  file=open(file_name),
  purpose="fine-tune"
)
f_t = openai.FineTune.create(
    training_file=f["id"],
    model="curie"
)

Upload progress: 100%|██████████| 250k/250k [00:00<00:00, 355kit/s]


In [85]:
model = get_model_by_dataset_name("generate-generalist-0.0.3.jsonl")
model

<FineTune fine-tune id=ft-eiaHkhbMgvOc3YL1SZL6bVV1 at 0x7fb3183235e0> JSON: {
  "created_at": 1628408249,
  "fine_tuned_model": "curie:ft-beaumont-2021-08-08-08-14-15",
  "hyperparams": {
    "batch_size": 4,
    "learning_rate_multiplier": 0.1,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1,
    "use_packing": null
  },
  "id": "ft-eiaHkhbMgvOc3YL1SZL6bVV1",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-6iVquBDkjuvNsgZOGPJsaRAs",
  "result_files": [
    {
      "bytes": 98682,
      "created_at": 1628410459,
      "filename": "compiled_results.csv",
      "id": "file-UM4nGDiiDj52YCkEXLp7mb1F",
      "model": null,
      "object": "file",
      "purpose": "fine-tune-results",
      "status": "processed",
      "status_details": null
    }
  ],
  "status": "succeeded",
  "training_files": [
    {
      "bytes": 249406,
      "created_at": 1628408248,
      "filename": "generate-generalist-0.0.3.jsonl",
      "id": "file-qNSpkQUrHjgQH6dhihh35f54",
      "model"

In [86]:
p = {}
p["type"] = "TopicGeneralistFineTuned3"
p["template"] = "[TOPIC]:"
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": get_model_by_dataset_name("generate-generalist-0.0.3.jsonl")["fine_tuned_model"],
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\n\n"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 8, 8, 8, 16, 3, 347757, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7fb2f2dbb2e0>)


In [104]:
openai.Completion.create(
  model=model["fine_tuned_model"],
  prompt="This a conversation starter about philosophy:",
  max_tokens=300,
  stop=["\n\n"]
)

<OpenAIObject text_completion id=cmpl-3UYLNZzAyutQOqqva47e9mkN4H3LC at 0x7fb3182db540> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": ""
    }
  ],
  "created": 1628410869,
  "id": "cmpl-3UYLNZzAyutQOqqva47e9mkN4H3LC",
  "model": "curie:ft-beaumont-2021-08-08-08-14-15",
  "object": "text_completion"
}

In [76]:
file_name = "classification-topic-endpoint-0.0.1.jsonl"
with open(file_name, 'w') as outfile:
    for e in langame_client._firestore_client.collection("memes").stream():
        json.dump({
            "text": e.get("content"),
            "label": e.get("topics")[0]
        }, outfile)
        outfile.write('\n')

In [77]:
c_f = openai.File.create(
    file=open(file_name),
    purpose="classifications"
)

Upload progress: 100%|██████████| 58.1k/58.1k [00:00<00:00, 104kit/s]


<File file id=file-rV9rZbiiziXLfjqc4MKLLChy at 0x7fb3187e4270> JSON: {
  "bytes": 57821,
  "created_at": 1628408105,
  "filename": "classification-topic-endpoint-0.0.1.jsonl",
  "id": "file-rV9rZbiiziXLfjqc4MKLLChy",
  "object": "file",
  "purpose": "classifications",
  "status": "uploaded",
  "status_details": null
}

In [4]:
openai.Classification.create(
    file="file-rV9rZbiiziXLfjqc4MKLLChy",
    query="123",
    search_model="ada", 
    model="curie", 
    max_examples=3
)

InvalidRequestError: No similar documents were found in file with ID 'file-rV9rZbiiziXLfjqc4MKLLChy'.Please upload more documents or adjust your query.